In [27]:
import pandas as pd
import seaborn as sns
import numpy as np

In [90]:
df = pd.read_excel('Copa-Data-File.xls')

In [91]:
df.head()

,Customer ID,Location,Booking date (# of days before the event),Type,Quote,Win
0,2007-10,3,174,Wedding,103,0
1,2007-100,4,174,Business,102,1
2,2007-1000,7,172,Wedding,141,0
3,2007-1001,9,36,Private,193,0
4,2007-1002,5,185,Wedding,141,1


In [92]:
df.columns = ['customer_id', 'location','date', 'type', 'quote', 'win']

In [93]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7155 entries, 0 to 7154
Data columns (total 6 columns):
customer_id    7155 non-null object
location       7155 non-null int64
date           7155 non-null int64
type           7155 non-null object
quote          7155 non-null int64
win            7155 non-null int64
dtypes: int64(4), object(2)
memory usage: 335.5+ KB


In [94]:
df['location'] = df['location'].astype('category')

In [95]:
df['type'] = df['type'].apply(str.strip)

In [96]:
df['type'].value_counts()

Wedding     2399
Private     2381
Business    2375
Name: type, dtype: int64

In [97]:
df['quote'].describe()

count    7155.000000
mean      150.444305
std        29.048393
min       100.000000
25%       125.000000
50%       151.000000
75%       176.000000
max       200.000000
Name: quote, dtype: float64

In [98]:
df.head()

,customer_id,location,date,type,quote,win
0,2007-10,3,174,Wedding,103,0
1,2007-100,4,174,Business,102,1
2,2007-1000,7,172,Wedding,141,0
3,2007-1001,9,36,Private,193,0
4,2007-1002,5,185,Wedding,141,1


In [110]:
df[df['win']==1]['quote'].describe()

count    2519.000000
mean      134.399762
std        25.494634
min       100.000000
25%       113.000000
50%       130.000000
75%       152.000000
max       199.000000
Name: quote, dtype: float64

In [100]:
type_dict = {'Wedding':1, 'Business':2, 'Private':3}

In [12]:
df['type'] = df['type'].map(type_dict).astype('category')

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [14]:
x = ['location', 'date', 'type', 'quote']

In [15]:
log1 = LogisticRegression()

In [16]:
X = df[x]
y = df['win']

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [44]:
X_train.shape

(5724, 4)

In [18]:
log1.fit(X_train, y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [19]:
# regression，不是classification
y_pred = log1.predict_proba(X_test)

In [20]:
X_test.iloc[1]

location      8
date        155
type          3
quote       121
Name: 3707, dtype: object

In [77]:
log1.classes_

array([0, 1])

In [78]:
y_pred[:,1]

array([0.09166416, 0.42922613, 0.14056852, ..., 0.37888182, 0.40980294,
       0.43798114])

In [22]:
# Object function: price * probablity
log1.coef_

array([[ 0.01130175, -0.01098676, -0.03196712, -0.03375764]])

In [23]:
loc_df = pd.read_excel('Copa-Data-File.xls', sheet_name='Sheet1')
loc_dict = dict(zip(loc_df['Location'],loc_df['Location-ID'], ))

In [24]:
loc_dict

{'Boston': 1,
 'NYC': 2,
 'Washington DC': 3,
 'Baltimore': 4,
 'Norfolk VA': 5,
 'Philadelphia': 6,
 'New Jersey': 7,
 'Seattle': 8,
 'National Harbor MD': 9,
 'Miami': 10}

In [112]:
# for each new customer
def give_quote(model, loc_dict, event_dict):
    # input variables
    loc = str(input('Please enter a location: '))
    date = int(input('Please enter the number of days to the event: '))
    event_type = str(input('Please enter the desired event type: '))
    lower = int(input('Please enter the lower limit of quote: '))
    upper = int(input('Please enter the upper limit of quote: '))
    # change variable types
    df = pd.DataFrame({'location':loc, 
                      'date':date,
                      'type':event_type}, index=range(1))
    df['location'] = df['location'].map(loc_dict).astype('category')
    df['type'] = df['type'].map(event_dict).astype('category')
    # calculate probablity under each quote
    exp_rev = []
    for price in range(lower,upper):
        x = np.hstack((df.values[0], price))
        p = model.predict_proba(x.reshape(-1,4))[:,1]
        exp_rev.append(float(price * p))
    
    rev_max = max(exp_rev)
    exp_price = [list(range(lower,upper))[index] for index, i in enumerate(exp_rev) if i == rev_max]
    print('Recommended Quote is: ')
    print(exp_price)

In [113]:
give_quote(log1, loc_dict, type_dict)

Please enter a location: Norfolk VA
Please enter the number of days to the event: 185
Please enter the desired event type: Wedding
Please enter the lower limit of quote: 100
Please enter the upper limit of quote: 200
Recommended Quote is: 
[100]


In [116]:
give_quote(log1, loc_dict, type_dict)

Please enter a location: Norfolk VA
Please enter the number of days to the event: 30
Please enter the desired event type: Wedding
Please enter the lower limit of quote: 0
Please enter the upper limit of quote: 200
Recommended Quote is: 
[121]
